This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X"
test_params = {
    "start_date": "2017-10-02",
    "end_date": "2017-10-02",
    "api_key": API_KEY
}
test_data = requests.get(url, params=test_params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
test_data.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-10-02',
  'end_date': '2017-10-02',
  'data': [['2017-10-02',
    44.51,
    44.98,
    44.18,
    44.98,
    None,
    95313.0,
    4265024.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Getting data for 2017

In [5]:
# construct and make the GET request
data_params = {
    "start_date": "2017-01-01",
    "end_date": "2017-12-31",
    "api_key": API_KEY
}

raw_data = requests.get(url, params=data_params).json()['dataset']

### 2. Parsing the data into a dict

Because of the odd restriction to not use pandas, we can parse the data into a `dict` of `dict`s such that the top-level keys are dates and the values are dicts where the keys are the column headings. Note that there is data only for days on which the exchange was open (255 in 2017).

In [6]:
column_names = raw_data["column_names"]
values_table = raw_data['data']
afx_x_data = {
    row[0]: {
        column_name: value
        for column_name, value in zip(column_names[1:], row[1:])
    }
    for row in values_table
}
print(f"Observations parsed: {len(afx_x_data)}")
print(f"Columns names: {column_names}")
print(f"data for Oct 2: {afx_x_data['2017-10-02']}")

Observations parsed: 255
Columns names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
data for Oct 2: {'Open': 44.51, 'High': 44.98, 'Low': 44.18, 'Close': 44.98, 'Change': None, 'Traded Volume': 95313.0, 'Turnover': 4265024.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}


### 3. The highest and lowest opening prices

In [7]:
# There are a few dates with no opening price. I filter those out.

max_open_date = max(
    filter(lambda date: afx_x_data[date]['Open'] is not None, afx_x_data),
    key=lambda day: afx_x_data[day]["Open"]
)
print(f"The highest opening price was on {max_open_date}, with a price of {afx_x_data[max_open_date]['Open']:.2f}.")
min_open_date = min(
    filter(lambda date: afx_x_data[date]['Open'] is not None, afx_x_data),
    key=lambda day: afx_x_data[day]["Open"]
)
print(f"The lowest opening price was on {min_open_date}, with a price of {afx_x_data[min_open_date]['Open']:.2f}.")

The highest opening price was on 2017-12-14, with a price of 53.11.
The lowest opening price was on 2017-01-24, with a price of 34.00.


### 4. Largest daily range

In [8]:
max_range_date = max(
    filter(
        lambda day: afx_x_data[day]['High'] is not None and afx_x_data[day]['Low'] is not None,
        afx_x_data
    ),
    key=lambda day: afx_x_data[day]['High'] - afx_x_data[day]['Low']
)
max_range = afx_x_data[max_range_date]['High'] - afx_x_data[max_range_date]['Low']
print(f"The highest range between high and low prices was {max_range:.2f} on {max_range_date}.")

The highest range between high and low prices was 2.81 on 2017-05-11.


### 5. Largest change between closing prices

This one requires a little interpretation: the question does not specifically mention *consecutive* days, but let's assume that is what is meant. Thankfully there are no missing values for close, so that simplifies things somewhat. There are gaps for days with no trading, however. For simplicitly sake, let's look for the maximum change between consecutive *trading* days. Also, let's assume "largest change" is independent of direction.

In [9]:
# no missing values for close
print([date for date in afx_x_data if afx_x_data[date]['Close'] is None])

[]


In [10]:
current_max_change = {"dates": None, "price_change": 0}
dates_in_order = sorted(afx_x_data.keys())
last_date = dates_in_order[0]
for date in dates_in_order[1:]:
    price_change =  afx_x_data[date]['Close'] - afx_x_data[last_date]['Close']
    if abs(price_change) > abs(current_max_change['price_change']):
        current_max_change = {
            "dates": f"{last_date} to {date}",
            "price_change": price_change
        }
    last_date = date

print(f"The largest day-to-day price change was {current_max_change['price_change']:.2f}, from {current_max_change['dates']}.")

The largest day-to-day price change was -2.56, from 2017-08-08 to 2017-08-09.


### 6. Average daily trading volume

In [11]:
# Check for missing values
print([date for date in afx_x_data if afx_x_data[date]['Traded Volume'] is None])

[]


In [12]:
# Make a list of daily trading volumes
daily_volumes = [obs['Traded Volume'] for obs in afx_x_data.values()]

# Take the average
avg_daily_volume = sum(daily_volumes) / len(daily_volumes)

print(f"The average daily trading volume (rounded to the nearest unit) was {avg_daily_volume:.0f}.")

The average daily trading volume (rounded to the nearest unit) was 89124.


### 7. Median daily trading volume

In [13]:
def median(vals):
    sorted_vals = sorted(vals)
    n = len(sorted_vals)
    if n % 2 == 0:
        return (sorted_vals[n/2 - 1] + sorted_vals[n/2])/2
    else:
        return sorted_vals[n//2]
    
median_daily_volume = median(daily_volumes)

print(f"The median daily trading volume was {median_daily_volume}.")

The median daily trading volume was 76286.0.
